In [2]:
!pip install --upgrade transformers
!pip install pytorchvideo evaluate -q
!pip install --upgrade datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 

In [3]:
audio_mdl = "minoosh/wav2vec2-base-finetuned-ie"
text_mdl = "minoosh/finetuned_bert-base-uncased"
video_mdl = "minoosh/videomae-base-finetuned-IEMOCAP_videos"

In [4]:
from huggingface_hub import login

login("???")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


# text classification example : pipeline and output_layer

In [ ]:
from datasets import load_dataset

dataset = load_dataset("minoosh/IEMOCAP_Text")

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
sample_text = ["guess who I am.", "be bad."]

In [ ]:
sample_text = dataset['Session5']['transcription'][:5]
sample_text

[' Hi, I need an ID.',
 " ahh Yeah, this is the wrong line.  I'm sorry.  You need to go back over to line two B.  That's where you should have started from.",
 " Well, I have no camera here.  I can't help you at all here, ma'am.  I have forms for something completely different.  You need to go to a different place.",
 ' So could I maybe get the form from you, fill it out here- and maybe you could-',
 " Isn't there anything you can do?  Just call over and ask them if I can go to the front of the line in the other line."]

In [ ]:
dataset['Session5']['emotion'][:5]

[2, 2, 2, 2, 0]

In [ ]:
from transformers import pipeline

classifier = pipeline(model=text_mdl)
classifier(sample_text)

[{'label': 'neu', 'score': 0.986386775970459},
 {'label': 'neu', 'score': 0.8787382245063782},
 {'label': 'neu', 'score': 0.9891164898872375},
 {'label': 'neu', 'score': 0.9975172281265259},
 {'label': 'neu', 'score': 0.9966309666633606}]

In [ ]:
# emotion : ang, hap, neu, sad
# (by order) 0, 1, 2, 3

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained(text_mdl)
inputs = tokenizer(sample_text, return_tensors="pt", padding=True, )

model = AutoModelForSequenceClassification.from_pretrained(text_mdl)
with torch.no_grad():
    logits = model(**inputs).logits

softmax_layer = torch.nn.Softmax(dim=1)
output = softmax_layer(logits)

In [ ]:
output 

tensor([[1.1136e-02, 7.6038e-04, 9.8639e-01, 1.7166e-03],
        [9.5639e-02, 2.3511e-02, 8.7874e-01, 2.1118e-03],
        [8.8632e-03, 5.3684e-04, 9.8912e-01, 1.4834e-03],
        [8.3920e-04, 8.7990e-04, 9.9752e-01, 7.6370e-04],
        [1.7367e-03, 5.0621e-04, 9.9663e-01, 1.1262e-03]])

# audio classification example : pipeline and output_layer

In [ ]:
from datasets import load_dataset

dataset = load_dataset("minoosh/IEMOCAP_Speech", use_auth_token=True)

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
sample_audio = dataset['Session5']['audio'][:5]

In [ ]:
dataset['Session5']['emotion'][:5]

[2, 2, 0, 0, 0]

In [ ]:
from transformers import pipeline

classifier = pipeline(model=audio_mdl)
classifier([samp['array'] for samp in sample_audio])

[[{'score': 0.8862055540084839, 'label': 'sad'},
  {'score': 0.05561116337776184, 'label': 'hap'},
  {'score': 0.03502825275063515, 'label': 'neu'},
  {'score': 0.02315502054989338, 'label': 'ang'}],
 [{'score': 0.9325020909309387, 'label': 'neu'},
  {'score': 0.031234214082360268, 'label': 'ang'},
  {'score': 0.019414938986301422, 'label': 'hap'},
  {'score': 0.016848795115947723, 'label': 'sad'}],
 [{'score': 0.8461610674858093, 'label': 'ang'},
  {'score': 0.11167491227388382, 'label': 'hap'},
  {'score': 0.0294108334928751, 'label': 'neu'},
  {'score': 0.012753215618431568, 'label': 'sad'}],
 [{'score': 0.9658694267272949, 'label': 'ang'},
  {'score': 0.015805210918188095, 'label': 'hap'},
  {'score': 0.01051913108676672, 'label': 'sad'},
  {'score': 0.007806210312992334, 'label': 'neu'}],
 [{'score': 0.8060486316680908, 'label': 'neu'},
  {'score': 0.10900448262691498, 'label': 'hap'},
  {'score': 0.0730169415473938, 'label': 'ang'},
  {'score': 0.011929923668503761, 'label': 'sad

In [ ]:
from transformers import AutoFeatureExtractor
from transformers import AutoModelForAudioClassification
import torch

feature_extractor = AutoFeatureExtractor.from_pretrained(audio_mdl)
inputs = feature_extractor([samp['array'] for samp in sample_audio], sampling_rate=16000, return_tensors="pt", padding=True)

model = AutoModelForAudioClassification.from_pretrained(audio_mdl)
with torch.no_grad():
    logits = model(**inputs).logits

softmax_layer = torch.nn.Softmax(dim=1)
output = softmax_layer(logits)

In [ ]:
output

tensor([[0.0105, 0.0663, 0.0446, 0.8786],
        [0.0480, 0.0233, 0.9133, 0.0155],
        [0.8462, 0.1117, 0.0294, 0.0128],
        [0.9636, 0.0189, 0.0080, 0.0096],
        [0.0921, 0.1311, 0.7654, 0.0114]])

# video classification example : pipeline and output_layer

In [5]:
from huggingface_hub import hf_hub_download


hf_dataset_identifier = "minoosh/IEMOCAP_videos"
filename = "IEMOCAP_videos"
file_path = hf_hub_download(
    repo_id=hf_dataset_identifier, filename=filename, repo_type="dataset"
)

In [6]:
!tar xf {file_path}

In [46]:
import pathlib

dataset_root_path = "IEMOCAP_videos"
dataset_root_path = pathlib.Path(dataset_root_path)
sample_video_file_paths = list(dataset_root_path.glob("Session5/*/*.mp4"))

In [47]:
video_samples = [str(item) for item in sample_video_file_paths]
video_samples.sort(key=lambda video_samples: video_samples[27:])

In [51]:
video_samples = video_samples[:5]
video_samples

['IEMOCAP_videos/Session5/neu/Ses05F_impro01_F000.mp4',
 'IEMOCAP_videos/Session5/neu/Ses05F_impro01_F008.mp4',
 'IEMOCAP_videos/Session5/ang/Ses05F_impro01_F017.mp4',
 'IEMOCAP_videos/Session5/ang/Ses05F_impro01_F018.mp4',
 'IEMOCAP_videos/Session5/ang/Ses05F_impro01_F020.mp4']

In [49]:
!pip install decord

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
from transformers import pipeline

classifier = pipeline(model=video_mdl)
classifier(video_samples)

[[{'score': 0.3742099404335022, 'label': 'sad'},
  {'score': 0.2603820264339447, 'label': 'neu'},
  {'score': 0.22082585096359253, 'label': 'hap'},
  {'score': 0.14458215236663818, 'label': 'ang'}],
 [{'score': 0.3368416726589203, 'label': 'sad'},
  {'score': 0.260442852973938, 'label': 'neu'},
  {'score': 0.24025654792785645, 'label': 'hap'},
  {'score': 0.16245891153812408, 'label': 'ang'}],
 [{'score': 0.3697992265224457, 'label': 'sad'},
  {'score': 0.25825437903404236, 'label': 'neu'},
  {'score': 0.2250988483428955, 'label': 'hap'},
  {'score': 0.14684748649597168, 'label': 'ang'}],
 [{'score': 0.27548453211784363, 'label': 'neu'},
  {'score': 0.2707561254501343, 'label': 'sad'},
  {'score': 0.26353719830513, 'label': 'hap'},
  {'score': 0.19022206962108612, 'label': 'ang'}],
 [{'score': 0.3397396206855774, 'label': 'sad'},
  {'score': 0.2574996054172516, 'label': 'neu'},
  {'score': 0.23963722586631775, 'label': 'hap'},
  {'score': 0.16312357783317566, 'label': 'ang'}]]

In [53]:
def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

In [54]:
import av
import torch
import numpy as np

from transformers import AutoFeatureExtractor, AutoModelForVideoClassification


feature_extractor = AutoFeatureExtractor.from_pretrained(video_mdl)
model = AutoModelForVideoClassification.from_pretrained(video_mdl)

softmax_layer = torch.nn.Softmax(dim=1)

for video_sample in video_samples:
  # video clip consists of 300 frames (10 seconds at 30 FPS)
  container = av.open(video_sample)

  # sample 16 frames
  indices = sample_frame_indices(clip_len=16, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
  video = read_video_pyav(container, indices)

  inputs = feature_extractor(list(video), return_tensors="pt")

  with torch.no_grad():
    logits = model(**inputs).logits
    output = softmax_layer(logits)
    print(output)

/usr/local/lib/python3.10/dist-packages/transformers/models/videomae/feature_extraction_videomae.py:28: FutureWarning: The class VideoMAEFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use VideoMAEImageProcessor instead.
  warnings.warn(


tensor([[0.1441, 0.2205, 0.2644, 0.3709]])
tensor([[0.2433, 0.2969, 0.2752, 0.1846]])
tensor([[0.1447, 0.2206, 0.2602, 0.3745]])
tensor([[0.2506, 0.3277, 0.2414, 0.1803]])
tensor([[0.1933, 0.2631, 0.2598, 0.2839]])
